版权声明：本材料是南开大学“python语言与机器学习”课程课件，版权归课程老师朱开恩所有。允许拷贝、分发使用，允许修改，但是请保留此版权声明。

版本时间：2020年9月

# 安装、例子

安装： sudo pip3 install pyqt5 vispy

下载例子  https://github.com/vispy/vispy  里面，点击clone or download 可以下载vispy代码，里面的examples目录有各种例子。

# 与jupyter-notebook的不兼容性

<font size=5 color='red'> 目前不兼容，请用py文件和命令行运行。 </font>

# 三维数据的显示技术: 从OpenGL到Vispy

## 使用matplotlib进行便捷的数据可视化
- numpy, scipy, matplotlib 作为科学计算和数据可视化的套件

- 但是当使用大量的数据点时性能就出现了问题

- 在进行三维数据绘制和显示的时候更会出现明显的卡顿

归结原因主要是因为matplotlib定位为高质量绘图, 并非高性能, 其采用CPU计算和绘制图形

## 使用GPU为可视化加速
![Logo](opengllogo.png)
GPU是专门为了图形显示为设计的处理器, 使用GPU可以运行大型3D游戏, 显示复杂而绚丽的光影特效等任务. 

In [2]:
!python3 atom.py  # 例子

In [15]:
!python3 dihedron_in_cube.py  # 例子

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


### OpenGL: 作为GPU编程的API接口
GPU虽然性能强大,但是想要使用却并不容易, 其中OpenGL就是一套图形API, 我们使用它进行复杂的3d图形编程, 虽然这并不是那么容易. 

### OpenGL: 如何让计算机显示图形
OpenGL是如何让计算机显示一个图形到显示器上? 比如一个三角形?

#### 显示应用窗口

#### OpenGL中的可编程管线
![pipeline](pipeline.png)

我们需要做的, 就是在图中**蓝色**的部分**输入需要的数据信息**, 然后OpenGL就会帮我们把图形画好显示到屏幕上, 这一切都将在显卡中完成, 速度非常之快.
需要输入的数据为:
1. 数据点坐标
```
float vertices[] = {
    -0.5f, -0.5f, 0.0f,
     0.5f, -0.5f, 0.0f,
     0.0f,  0.5f, 0.0f
};
```
2. shader(着色器)代码
    - 顶点着色器(Vertex Shader): 处理数据点在空间中的坐标
    ```
    #version 330 core
    layout (location = 0) in vec3 aPos;
    ...
    uniform mat4 model;
    uniform mat4 view;
    uniform mat4 projection;

    void main()
    {
        // 注意乘法要从右向左读
        gl_Position = projection * view * model * vec4(aPos, 1.0);
        ...
    }
    ```
    - 片段着色器(Fragment Shader): 处理栅格化之后的每个小片的颜色
    ```
    #version 330 core
    out vec4 FragColor;

    void main()
    {
        FragColor = vec4(1.0f, 0.5f, 0.2f, 1.0f);
    } 
    ```

#### 坐标系统
OpenGL希望在每次顶点着色器运行后，我们可见的所有顶点都为标准化设备坐标(Normalized Device Coordinate, NDC)。也就是说，每个顶点的x，y，z坐标都应该在-1.0到1.0之间，超出这个坐标范围的顶点都将不可见.

所以我们需要将原始的坐标点进行变化.

对于3D的向量, 要将其在三维空间中进行缩放, 旋转和平移的操作, 我们需要使用4X4的仿射矩阵. 人为地, 我们将3d数据的变换分为三步, 每一步对应一个仿射矩阵:
![coord](coordinate_systems.png)
其中:
- model: 将局部坐标移至世界坐标


- view:  将世界坐标转化为观察者的坐标(摄像机)


- projection: 将3d世界投影到2d的屏幕上

# 坐标变换
按 transforms.pdf 讲解

# Vispy: 绑定Python的OpenGL的高级接口

## Vispy做了什么:

- 将原本是用C/C++语言开发的OpenGL用Python包起来让我们可以方便的用Python开发高性能图形应用, (GLOO低级接口)
- 提供了高级接口, 像使用matplotlib一样方便, (scene)

## vispy画图的例子

In [1]:
# 复杂的粒子效果
!python3 atom.py

In [2]:
# 丰富的交互和特效
!python3 rain.py

rain.py:98: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('a_size',     np.float32, 1)])


In [3]:
# 大量分子显示
!python3 molecular_viewer.py

molecular_viewer.py:141: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('a_radius', np.float32, 1)])


## 高级接口Scene画图

- 使用高级接口的例子, Scene

### 例子1
x-z平面的项链，可以缩放、旋转

In [4]:
!python3 basic01.py

In [5]:
# %load basic01.py
# vispy==0.4.0dev
"""
基础例子
"""
import sys
from vispy import scene, app
import numpy as np

# 数据生成
N = 100
theta = np.linspace(0, 2* np.pi, N)
r = 1.0
x = r* np.sin(theta)
z = r* np.cos(theta)
y = np.zeros(N)

pos = np.array([x,y,z]).T


# Vispy 程序骨架
canvas = scene.SceneCanvas(title='basic01', keys='interactive', show=True)

viewbox = canvas.central_widget.add_view()
viewbox.camera = 'turntable'

axis = scene.visuals.XYZAxis(parent=viewbox.scene)

scatter = scene.visuals.Markers() #散点
viewbox.add(scatter)
scatter.set_data(pos)

line1 = scene.visuals.Line() # 线
viewbox.add(line1)
line1.set_data(pos)

if __name__ == '__main__' and sys.flags.interactive == 0:
    app.run()


### 例子2

In [7]:
!python3 line_plot3d.py

In [8]:
# pyline: disable=no-member
""" plot3d using existing visuals : LinePlotVisual """

import numpy as np
import sys

from vispy import app, visuals, scene

# build visuals
Plot3D = scene.visuals.create_visual_node(visuals.LinePlotVisual)

# build canvas
canvas = scene.SceneCanvas(keys='interactive', title='plot3d', show=True)

# Add a ViewBox to let the user zoom/rotate
view = canvas.central_widget.add_view()
view.camera = 'turntable'
view.camera.fov = 45
view.camera.distance = 6

# prepare data
N = 60
x = np.sin(np.linspace(-2, 2, N)*np.pi)
y = np.cos(np.linspace(-2, 2, N)*np.pi)
z = np.linspace(-2, 2, N)

# plot
pos = np.c_[x, y, z]
Plot3D(pos, width=2.0, color='red',
       edge_color='w', symbol='o', face_color=(0.2, 0.2, 1, 0.8),
       parent=view.scene)


if __name__ == '__main__':
    if sys.flags.interactive != 1:
        app.run()

## 用GLoo低级接口画图

- 一个使用GLoo低级接口的例子, 旋转的立方体

In [4]:
# 旋转的立方体
!python3 rotate_cube.py

In [10]:
# %load rotate_cube.py
#!/usr/bin/env python
# vispy: gallery 50
"""
This example shows how to display 3D objects.
You should see a colored outlined spinning cube.
"""

import numpy as np
from vispy import app, gloo
from vispy.util.transforms import perspective, translate, rotate

#### 着色器代码

##### 顶点着色器
vert = """
// Uniforms
// ------------------------------------
uniform   mat4 u_model;
uniform   mat4 u_view;
uniform   mat4 u_projection;
uniform   vec4 u_color;

// Attributes
// ------------------------------------
attribute vec3 a_position;
attribute vec4 a_color;
attribute vec3 a_normal;

// Varying
// ------------------------------------
varying vec4 v_color;

void main()
{
    v_color = a_color * u_color;
    gl_Position = u_projection * u_view * u_model * vec4(a_position,1.0);
}
"""

##### 片段着色器
frag = """
// Varying
// ------------------------------------
varying vec4 v_color;

void main()
{
    gl_FragColor = v_color;
}
"""


# -----------------------------------------------------------------------------
def cube():
    """
    Build vertices for a colored cube.

    V  is the vertices
    I1 is the indices for a filled cube (use with GL_TRIANGLES)
    I2 is the indices for an outline cube (use with GL_LINES)
    """
    vtype = [('a_position', np.float32, 3),
             ('a_normal', np.float32, 3),
             ('a_color', np.float32, 4)]
    # Vertices positions
    v = [[1, 1, 1], [-1, 1, 1], [-1, -1, 1], [1, -1, 1],
         [1, -1, -1], [1, 1, -1], [-1, 1, -1], [-1, -1, -1]]
    # Face Normals
    n = [[0, 0, 1], [1, 0, 0], [0, 1, 0],
         [-1, 0, 1], [0, -1, 0], [0, 0, -1]]
    # Vertice colors
    c = [[0, 1, 1, 1], [0, 0, 1, 1], [0, 0, 0, 1], [0, 1, 0, 1],
         [1, 1, 0, 1], [1, 1, 1, 1], [1, 0, 1, 1], [1, 0, 0, 1]]

    V = np.array([(v[0], n[0], c[0]), (v[1], n[0], c[1]),
                  (v[2], n[0], c[2]), (v[3], n[0], c[3]),
                  (v[0], n[1], c[0]), (v[3], n[1], c[3]),
                  (v[4], n[1], c[4]), (v[5], n[1], c[5]),
                  (v[0], n[2], c[0]), (v[5], n[2], c[5]),
                  (v[6], n[2], c[6]), (v[1], n[2], c[1]),
                  (v[1], n[3], c[1]), (v[6], n[3], c[6]),
                  (v[7], n[3], c[7]), (v[2], n[3], c[2]),
                  (v[7], n[4], c[7]), (v[4], n[4], c[4]),
                  (v[3], n[4], c[3]), (v[2], n[4], c[2]),
                  (v[4], n[5], c[4]), (v[7], n[5], c[7]),
                  (v[6], n[5], c[6]), (v[5], n[5], c[5])],
                 dtype=vtype)
    I1 = np.resize(np.array([0, 1, 2, 0, 2, 3], dtype=np.uint32), 6 * (2 * 3))
    I1 += np.repeat(4 * np.arange(2 * 3, dtype=np.uint32), 6)

    I2 = np.resize(
        np.array([0, 1, 1, 2, 2, 3, 3, 0], dtype=np.uint32), 6 * (2 * 4))
    I2 += np.repeat(4 * np.arange(6, dtype=np.uint32), 8)

    return V, I1, I2


# -----------------------------------------------------------------------------
class Canvas(app.Canvas):

    def __init__(self):
        app.Canvas.__init__(self, keys='interactive', size=(800, 600))
        
        # 数据绑定
        self.vertices, self.filled, self.outline = cube()
        self.filled_buf = gloo.IndexBuffer(self.filled)
        self.outline_buf = gloo.IndexBuffer(self.outline)
        
        # 着色器绑定
        self.program = gloo.Program(vert, frag)
        self.program.bind(gloo.VertexBuffer(self.vertices))
        
        # 变换矩阵的生成和绑定
        self.view = translate((0, 0, -5))
        self.model = np.eye(4, dtype=np.float32)

        gloo.set_viewport(0, 0, self.physical_size[0], self.physical_size[1])
        self.projection = perspective(45.0, self.size[0] /
                                      float(self.size[1]), 2.0, 10.0)

        self.program['u_projection'] = self.projection

        self.program['u_model'] = self.model
        self.program['u_view'] = self.view

        self.theta = 0
        self.phi = 0

        gloo.set_clear_color('white')
        gloo.set_state('opaque')
        gloo.set_polygon_offset(1, 1)

        self._timer = app.Timer('auto', connect=self.on_timer, start=True)

        self.show()
    
    # 计时器函数回调, 让立方体动起来
    # ---------------------------------
    def on_timer(self, event):
        self.theta += .5
        self.phi += .5
        self.model = np.dot(rotate(self.theta, (0, 1, 0)),
                            rotate(self.phi, (0, 0, 1)))
        self.program['u_model'] = self.model
        self.update()

    # ---------------------------------
    def on_resize(self, event):
        gloo.set_viewport(0, 0, event.physical_size[0], event.physical_size[1])
        self.projection = perspective(45.0, event.size[0] /
                                      float(event.size[1]), 2.0, 10.0)
        self.program['u_projection'] = self.projection
    
    # 每一帧的绘制函数
    # ---------------------------------
    def on_draw(self, event):
        gloo.clear()

        # Filled cube

        gloo.set_state(blend=False, depth_test=True, polygon_offset_fill=True)
        self.program['u_color'] = 1, 1, 1, 1
        self.program.draw('triangles', self.filled_buf)

        # Outline
        gloo.set_state(blend=True, depth_test=True, polygon_offset_fill=False)
        gloo.set_depth_mask(False)
        self.program['u_color'] = 0, 0, 0, 1
        self.program.draw('lines', self.outline_buf)
        gloo.set_depth_mask(True)


# -----------------------------------------------------------------------------
if __name__ == '__main__':
    c = Canvas()
    app.run()


## 计时器与动画

- 小练习: 螺旋线
展示如何使用计时器(动画), 摄像机, 按键相应

In [6]:
!python3 ex1_start.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [ ]:
# vispy==0.4.0dev
"""
基础例子
"""
import sys
from vispy import scene, app
import numpy as np

# 数据生成
N = 100
theta = np.linspace(0, 6* np.pi, N)
r = 0.5
x = r* np.sin(theta)
z = r* np.cos(theta)
#y = np.zeros(N)
y = np.linspace(0, 1, N)

pos = np.array([x,y,z]).T


# Vispy 程序骨架
canvas = scene.SceneCanvas(title='basic01', keys='interactive', show=True)

viewbox = canvas.central_widget.add_view()
viewbox.camera = 'fly'

axis = scene.visuals.XYZAxis(parent=viewbox.scene)

scatter = scene.visuals.Markers() #散点
viewbox.add(scatter)
pos1 = np.array([pos[0]])
scatter.set_data(pos1)

#line1 = scene.visuals.Line() # 线
# viewbox.add(line1)
#line1.set_data(pos)


line1 = scene.visuals.Line(pos=pos, parent=viewbox.scene) # 线

count = 0
def on_timer(event):
    global count
    count += 1
    count = count % N 
    scatter.set_data(np.array([pos[count]]))
    


timer = app.Timer(connect=on_timer, start=True)
#timer.connect(on_timer)
#timer.start()

Running = True
@canvas.events.key_press.connect
def on_key(event):
    global Running
    if event.key == 'Space':
        if Running:
            timer.stop()
        else:
            timer.start()
        Running= not Running



if __name__ == '__main__' and sys.flags.interactive == 0:
    app.run()

## 点线面体

- 更多丰富的高级接口, 点线面体的显示

In [7]:
!python3 dihedron_in_cube.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [ ]:
import sys

from vispy import scene
from vispy.color import Color

import numpy as np


canvas = scene.SceneCanvas(keys='interactive', size=(800, 600), show=True)

# Set up a viewbox to display the cube with interactive arcball
view = canvas.central_widget.add_view()
#view.bgcolor = '#efefef'
#view.bgcolor = '#ffffff'
view.camera = 'turntable' # 摄像机
view.padding = 5

#color = Color("#3f51b5")
color = Color("#20af20")

nv = 24
vcolor = np.ones((nv, 4), dtype=np.float32)
rng = np.random.RandomState(1)
vcolor[:, 0] = np.linspace(1, 0, nv)
vcolor[:, 1] = rng.randn(nv)
vcolor[:, 2] = np.linspace(0, 1, nv)
vcolor[:, 3] = 0.8

cube = scene.visuals.Cube(size=1, vertex_colors=vcolor, edge_color="black", parent=view.scene) # 立方体
#cube = scene.visuals.Cube(size=1, color=color, edge_color="black", parent=view.scene)
cube.mesh.set_gl_state('additive', blend=True, depth_test=False)


pos = np.array([[1,-1,1],[-1,1,1],
                [1,1,-1],[1,-1,1] ], 
        dtype=np.float32) 
line = scene.visuals.Line(pos=pos, color='brown', parent=view.scene, width=2.0) # 线1

pos2 = np.array([[1,-1,-1],[-1,-1,1],
                [-1,1,-1],[1,-1,-1] ], 
        dtype=np.float32) 
line2 = scene.visuals.Line(pos=pos2, color='pink', parent=view.scene, width=2.0) # 线2

def get_one_third(a,b):
    return a+(b-a)/3.

pos3 = np.array([get_one_third(pos[0], pos[1]),
                 get_one_third(pos[1], pos[2]),
                 get_one_third(pos[2], pos[3]), ], 
        dtype=np.float32) 

pos4 = np.array([get_one_third(pos2[1], pos2[0]),
                 get_one_third(pos2[2], pos2[1]),
                 get_one_third(pos2[3], pos2[2]), ], 
        dtype=np.float32) 
face2 = scene.visuals.Mesh(vertices=pos4, color='pink', parent=view.scene) # 面2

#face1 = scene.visuals.Mesh(vertices=pos3, color='brown', parent=view.scene)
face1 = scene.visuals.Mesh(vertices=pos3, color='#40A0F080', parent=view.scene) # 面1
face1.set_gl_state('additive', blend=True, depth_test=False)

pos_middle = np.array([[1,-1,0],[0,-1,1],
                [-1,0,1],[-1,1,0],
                [0,1,-1], [1,0,-1],
                [1,-1,0] ], 
        dtype=np.float32) 
#line_middle = scene.visuals.Line(pos=pos_middle, color='blue', parent=view.scene)

pos_between1 = np.array([pos3[0], pos4[0]])
l1 = scene.visuals.Tube(pos_between1, color=['red', 'green', 'yellow'], shading='smooth', tube_points=8, radius=0.02, parent=view.scene)

pos_between2 = np.array([pos3[1], pos4[1]])
l2 = scene.visuals.Tube(pos_between2, color=['red', 'green', 'yellow'], shading='smooth', tube_points=8, radius=0.02, parent=view.scene)

pos_between3 = np.array([pos3[2], pos4[2]])
l3 = scene.visuals.Tube(pos_between3, color=['red', 'green', 'yellow'], shading='smooth', tube_points=8, radius=0.02, parent=view.scene) # 管

#axis = scene.visuals.XYZAxis(width=2.0, parent=view.scene)
#axis.set_data(pos=axis.pos*3)

if __name__ == '__main__' and sys.flags.interactive == 0:
    canvas.app.run()

## 也可以画2D！

In [8]:
!python3 simple_line.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [ ]:
import numpy as np
from vispy import plot as vp

t = np.linspace(0,2,201)
x = np.cos(2 * np.pi * t)
y = (t-1)**2

fig = vp.Fig(size=(600, 800), show=False)
fig[0,0].plot(np.array((t,x)).T, marker_size=0)
fig[1,0].plot(np.array((t,y)).T, marker_size=0)
fig.show(run=True)

In [9]:
!python3 plot.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [ ]:
import numpy as np

from vispy import plot as vp

fig = vp.Fig(size=(600, 500), show=False)

# Plot the target square wave shape
x = np.linspace(0, 10, 1000)
y = np.zeros(1000)
y[1:500] = 1
y[500:-1] = -1
line = fig[0, 0].plot((x, y), width=3, color='k',
                      title='Square Wave Fourier Expansion', xlabel='x',
                      ylabel='4/π Σ[ 1/n sin(nπx/L) | n=1,3,5,...]')

y = np.zeros(1000)
L = 5
colors = [(0.8, 0, 0, 1),
          (0.8, 0, 0.8, 1),
          (0, 0, 1.0, 1),
          (0, 0.7, 0, 1), ]
plot_nvals = [1, 3, 7, 31]
for i in range(16):
    n = i * 2 + 1
    y += (4. / np.pi) * (1. / n) * np.sin(n * np.pi * x / L)
    if n in plot_nvals:
        tmp_line = fig[0, 0].plot((x, y), color=colors[plot_nvals.index(n)],
                                  width=2)
        tmp_line.update_gl_state(depth_test=False)

labelgrid = fig[0, 0].view.add_grid(margin=10)

box = vp.Widget(bgcolor=(1, 1, 1, 0.6), border_color='k')
box_widget = labelgrid.add_widget(box, row=0, col=1)
box_widget.width_max = 90
box_widget.height_max = 120

bottom_spacer = vp.Widget()
labelgrid.add_widget(bottom_spacer, row=1, col=0)

labels = [vp.Label('n=%d' % plot_nvals[i], color=colors[i], anchor_x='left')
          for i in range(len(plot_nvals))]
boxgrid = box.add_grid()
for i, label in enumerate(labels):
    label_widget = boxgrid.add_widget(label, row=i, col=0)


grid = vp.visuals.GridLines(color=(0, 0, 0, 0.5))
grid.set_gl_state('translucent')
fig[0, 0].view.add(grid)


if __name__ == '__main__':
    fig.show(run=True)

# 要求掌握的例子

## 画线1
simple_line.py

In [10]:
!python3 simple_line.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


## 画线2

In [11]:
!python3 basic01.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [12]:
!python3 ex1_final.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


## 画线3

In [13]:
!python3 line_plot3d.py

/usr/local/lib/python3.6/dist-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


# 练习题

## 画出一个向右行进的正弦波

## 画出一个旋转的螺旋线